In [2]:
# require at least Python 3.5 for async/await to work
import sys
ver=sys.version_info
assert (ver[0]>=3 and ver[1]>=5)
sys.path.append('../src')
sys.path.append('../../msq-domain/src')
sys.path.append('../../quant_container/src')

import aiostreams.operators as op
from aiostreams.config import QCConfigProvider
# this is where AsyncKafkaSource, AsyncKafkaPublisher get the Kafka broker location
QCConfigProvider().kafka_broker = 'kafka:9092'

print(QCConfigProvider().kafka_broker)

kafka:9092


In [3]:
# try writing just one record to Kafka using the basic kafka library, 
# and reading back from it, to make sure Kafka is up, 
from kafka import KafkaProducer, KafkaConsumer
import pickle

topic='test_topic_2'
kafka_host = QCConfigProvider().kafka_broker

kprod=KafkaProducer(bootstrap_servers=kafka_host)
kprod.send(topic,pickle.dumps('test'))

consumer = KafkaConsumer(topic,bootstrap_servers=kafka_host,auto_offset_reset='earliest',
                         group_id=None, value_deserializer=lambda x: pickle.loads(x))
next(consumer).value

'test'

In [18]:
# let's check the dump was successful
from aiostreams import AsyncKafkaSource, run
import aiostreams.operators as op
from mosaicsmartdata.common.json_convertor import *
import json

input_graph = AsyncKafkaSource('bond-quotes-topic', \
                               value_deserializer =  lambda x: x.decode('utf-8'))|\
                            op.map(lambda x: json_to_domain(x.value).timestamp) > print # 
                           #op.map(lambda x: json.loads(x.value)['marketDataSnapshotFullRefreshList'][0]['timestamp']) > print #op.map(lambda x: json_to_domain(x.value).timestamp) > print #op.map(lambda x: x.value ) > print
                           # op.map(lambda x: json.loads(x.value) ) > print# 
                           #op.map(lambda x: json.loads(x.value) ) > print# 
                #
run(input_graph)

In [17]:
# let's check the dump was successful
from aiostreams import AsyncKafkaSource, run
import aiostreams.operators as op
from mosaicsmartdata.common.json_convertor import *
import json

input_graph = AsyncKafkaSource('bond-trades-topic', \
                               value_deserializer =  lambda x: x.decode('utf-8'))| \
                                op.map(lambda x: json_to_domain(x.value).timestamp) > print#
                               #op.map(lambda x: print(x.value) > print# #op.map(lambda x: json.loads(x.value) ) > []                            
    # #op.map(lambda x: json.loads(x.value) ) > []
                           #
                            #
                #
                
run(input_graph)
# print(input_graph.sink[0].value)
# json_to_domain(input_graph.sink[0].value)

In [14]:
# Look at what QC is outputting inside docker-compose
from aiostreams import AsyncKafkaSource, run
import aiostreams.operators as op
from mosaicsmartdata.common.json_convertor import *
import json

input_graph = AsyncKafkaSource('output-topic',\
                               value_deserializer =  lambda x: x.decode('utf-8'))|\
                op.map(lambda x: x.value ) > print
                #op.map(lambda x: json_to_domain(x.value)) > print
                #op.map(lambda x: json.loads(x.value) ) > print
                
run(input_graph)

In [16]:
# Let's load a snapshot of the graph locally and run against the same data
from aiostreams import AsyncKafkaSource, AsyncKafkaPublisher, run
import aiostreams.operators as op
from aiostreams.persist import KafkaPersister
from mosaicsmartdata.common.json_convertor import *
import json

graph_name = 'test_pid__markouts_unhedged' # got that name from the logs
p = KafkaPersister()
reflated_graph = p.load(graph_name)
reflated_graph.persistence_policy = None # don't want to persist states when we re-run
reflated_graph.sink = print #so we don't pollute the original output topic
                
run(reflated_graph)

AttributeError: 'NoneType' object has no attribute 'persistence_policy'